## Data Prep

**Import necessary modules and the ASOS/SNOTEL dataframes**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dtb
import os
from glob import glob
import datetime as dt
import seaborn as sns

In [2]:
data = [pd.read_csv('asos_snotel_clean_w_LXV.dat', parse_dates = True, index_col = 'Date_Time')]
asos_snotel_df= pd.concat(data)


#asos_snotel_df = asos_snotel_df.asfreq('1H')

#asos_snotel_df = asos_snotel_df.interpolate(limit=3)
#asos_snotel_df['Julian'] = asos_snotel_df.index.to_julian_date()
asos_snotel_df = asos_snotel_df.astype('float32')

print(asos_snotel_df.head())



                           CMtnSNTL_Temp_degC  CMtnSNTL_SnowDepth_in  \
Date_Time                                                              
2006-01-01 01:00:00+00:00                -1.3                   43.0   
2006-01-01 02:00:00+00:00                -2.7                   43.0   
2006-01-01 03:00:00+00:00                -2.6                   43.0   
2006-01-01 04:00:00+00:00                -2.5                   43.0   
2006-01-01 05:00:00+00:00                -3.2                   42.0   

                           CMtn_Temperature_degC  CMtn_Dewpoint_degC  \
Date_Time                                                              
2006-01-01 01:00:00+00:00                   -1.0                -7.0   
2006-01-01 02:00:00+00:00                   -3.0                -8.0   
2006-01-01 03:00:00+00:00                   -4.0                -8.0   
2006-01-01 04:00:00+00:00                   -4.0                -9.0   
2006-01-01 05:00:00+00:00                   -5.0               

#### Import KGJT and KDEN Upper Air Data

In [3]:
upper_data = None
upper_data_df = None
UA_df = None
UA_KGJT_df = None
UA_KDEN_df = None


UA_df = pd.DataFrame()
UA_KGJT = [pd.read_csv('KGJT_2006010100_2018010100.txt', header= 0, parse_dates = ['validUTC'], index_col = 'validUTC')]
UA_KDEN= [pd.read_csv('KDEN_2005010100_2018010100.txt', header= 0, parse_dates = ['validUTC'], index_col = 'validUTC')]




#print(UA_KDEN)

#upper_data = [pd.read_csv('KGJT_2006010100_2018010100.txt', header= 0)]
UA_KGJT_df = pd.concat(UA_KGJT)
print(UA_KGJT_df.head())

del UA_KGJT_df['station']
del UA_KGJT_df['bearing']
del UA_KGJT_df['range_sm']




UA_KGJT_df = UA_KGJT_df.query('levelcode == 4')
UA_KGJT_df = UA_KGJT_df.replace('M', np.NaN)
UA_KGJT_df = UA_KGJT_df.apply(pd.to_numeric)


UA_KGJT_df = UA_KGJT_df.astype('float32')
UA_KGJT_df.index.names = ['Date_Time']
#UA_KGJT_df.rename(index={'validUTC':'Date_Time'}, inplace = True)
#UA_KGJT_df = UA_KGJT_df.add_prefix('KGJT_')



UA_KDEN_df = pd.concat(UA_KDEN)
print(UA_KDEN_df.head())

#nont useful column and saves memory
del UA_KDEN_df['station'] 
del UA_KDEN_df['bearing'] 
del UA_KDEN_df['range_sm']

UA_KDEN_df = UA_KDEN_df.query('levelcode == 4')
UA_KDEN_df = UA_KDEN_df.replace('M', np.NaN)
UA_KDEN_df = UA_KDEN_df.apply(pd.to_numeric)
UA_KDEN_df = UA_KDEN_df.astype('float32')
UA_KDEN_df.index.names = ['Date_Time']
#UA_KDEN_df = UA_KDEN_df.rename(index={'validUTC':'Date_Time'})
#UA_KDEN_df = UA_KDEN_df.add_prefix('KDEN_')


print(UA_KGJT_df.head())
print(UA_KDEN_df.head())


#


c:\users\rapp\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                    station  levelcode  pressure_mb height_m tmpc dwpc drct  \
validUTC                                                                      
2006-12-03 12:00:00    KGJT          4       1000.0    370.0    M    M    M   
2006-12-03 12:00:00    KGJT          4        925.0    988.0    M    M    M   
2006-12-27 12:00:00    KGJT          4        850.0   1463.0    M    M    M   
2006-12-05 00:00:00    KGJT          4        925.0    934.0    M    M    M   
2006-12-23 12:00:00    KGJT          4       1000.0    273.0    M    M    M   

                    speed_kts bearing range_sm  
validUTC                                        
2006-12-03 12:00:00         M       M        M  
2006-12-03 12:00:00         M       M        M  
2006-12-27 12:00:00         M       M        M  
2006-12-05 00:00:00         M       M        M  
2006-12-23 12:00:00         M       M        M  
           station  levelcode  pressure_mb height_m tmpc  dwpc drct speed_kts  \
validUTC              

Read in most useful levels and create columns which calculate differences between different levels

In [4]:

levels = [200, 250, 300, 400, 500, 700, 850]

UA_copy_df = UA_KGJT_df.copy()
asos_snotel_UA_df = asos_snotel_df.copy() 
UA_KGJT_df.index = UA_KGJT_df.index.tz_localize('UTC')

for level in levels:
  print(level)
  asos_snotel_UA_df = asos_snotel_UA_df.join(UA_KGJT_df[UA_KGJT_df['pressure_mb'] == level].add_prefix('KGJT_'+str(level)+'mb_'), how= 'outer')

#print(test_df.describe())
print(asos_snotel_UA_df.describe())
#print(upper_data[upper_data_df['pressure_mb'][:] == 850] - upper_data_df[upper_data_df['pressure_mb'][:] == 500])

pairs = [[850, 700], [850, 500], [850, 400], [850, 300], [850, 250], [850, 200], \
         [700, 500], [700, 400], [700, 300], [700, 250], [700, 200], \
         [500, 400], [500, 300], [500,250], [500,200], \
         [400, 300], [400,250], [400,200], \
         [300, 250], [300, 200], \
         [250, 200]]
         
for pair in pairs:
 asos_snotel_UA_df = asos_snotel_UA_df.join(((UA_KGJT_df[UA_KGJT_df['pressure_mb'] == pair[0]]) - (UA_KGJT_df[UA_KGJT_df['pressure_mb'] == pair[1]])).add_prefix('KGJT_d'+str(pair[0])+'_'+str(pair[1])+'_'))

print(asos_snotel_UA_df.describe())
print(asos_snotel_UA_df.head())
print(asos_snotel_UA_df.keys())





200
250
300
400
500
700
850
       CMtnSNTL_Temp_degC  CMtnSNTL_SnowDepth_in  CMtn_Temperature_degC  \
count       104502.000000           97218.000000           79473.000000   
mean             1.321650              20.144232               1.521068   
std              9.144908              20.979582               9.262988   
min            -33.099998              -3.000000             -28.000000   
25%             -5.100000               0.000000              -5.000000   
50%              1.300000              15.000000               1.000000   
75%              7.730000              39.000000               9.000000   
max             25.100000              74.000000              24.000000   

       CMtn_Dewpoint_degC  CMtn_WindDirection_deg  CMtn_WindSpeed_mps  \
count        79451.000000            76568.000000        76568.000000   
mean            -8.996525              230.807068            6.139158   
std              8.146396               72.085175            3.443908   
min 

In [6]:
UA_KDEN_df.index = UA_KDEN_df.index.tz_localize('UTC')
for level in levels:
  print(level)
  asos_snotel_UA_df = asos_snotel_UA_df.join(UA_KDEN_df[UA_KDEN_df['pressure_mb'] == level].add_prefix('KDEN_'+str(level)+'mb_'), how= 'outer')
  
    
#print(test_df.describe())
print(asos_snotel_UA_df.describe())
#print(upper_data[upper_data_df['pressure_mb'][:] == 850] - upper_data_df[upper_data_df['pressure_mb'][:] == 500])

pairs = [[850, 700], [850, 500], [850, 400], [850, 300], [850, 250], [850, 200], \
         [700, 500], [700, 400], [700, 300], [700, 250], [700, 200], \
         [500, 400], [500, 300], [500,250], [500,200], \
         [400, 300], [400,250], [400,200], \
         [300, 250], [300, 200], \
         [250, 200]]
for pair in pairs:
 asos_snotel_UA_df = asos_snotel_UA_df.join(((UA_KDEN_df[UA_KDEN_df['pressure_mb'] == pair[0]]) - (UA_KDEN_df[UA_KDEN_df['pressure_mb'] == pair[1]])).add_prefix('KDEN_d'+str(pair[0])+'_'+str(pair[1])+'_'))

print(asos_snotel_UA_df.describe())
print(asos_snotel_UA_df.head())
print(asos_snotel_UA_df.keys())


200
250
300
400
500
700
850
       CMtnSNTL_Temp_degC  CMtnSNTL_SnowDepth_in  CMtn_Temperature_degC  \
count       104502.000000           97218.000000           79473.000000   
mean             1.321650              20.144232               1.521068   
std              9.144908              20.979582               9.262988   
min            -33.099998              -3.000000             -28.000000   
25%             -5.100000               0.000000              -5.000000   
50%              1.300000              15.000000               1.000000   
75%              7.730000              39.000000               9.000000   
max             25.100000              74.000000              24.000000   

       CMtn_Dewpoint_degC  CMtn_WindDirection_deg  CMtn_WindSpeed_mps  \
count        79451.000000            76568.000000        76568.000000   
mean            -8.996525              230.807068            6.139158   
std              8.146396               72.085175            3.443908   
min 

In [7]:
#Remove Unnecessary columns
asos_snotel_UA_df = asos_snotel_UA_df[asos_snotel_UA_df.columns.drop(list(asos_snotel_UA_df.filter(regex='levelcode')))]
asos_snotel_UA_df = asos_snotel_UA_df[asos_snotel_UA_df.columns.drop(list(asos_snotel_UA_df.filter(regex='pressure_mb')))]
asos_snotel_UA_df = asos_snotel_UA_df[asos_snotel_UA_df.columns.drop(list(asos_snotel_UA_df.filter(regex='KDEN')))]

print(asos_snotel_UA_df.describe())
print(asos_snotel_UA_df.info())
print(asos_snotel_UA_df.head())

       CMtnSNTL_Temp_degC  CMtnSNTL_SnowDepth_in  CMtn_Temperature_degC  \
count       104502.000000           97218.000000           79473.000000   
mean             1.321650              20.144232               1.521068   
std              9.144908              20.979582               9.262988   
min            -33.099998              -3.000000             -28.000000   
25%             -5.100000               0.000000              -5.000000   
50%              1.300000              15.000000               1.000000   
75%              7.730000              39.000000               9.000000   
max             25.100000              74.000000              24.000000   

       CMtn_Dewpoint_degC  CMtn_WindDirection_deg  CMtn_WindSpeed_mps  \
count        79451.000000            76568.000000        76568.000000   
mean            -8.996525              230.807068            6.139158   
std              8.146396               72.085175            3.443908   
min            -45.000000       

In [8]:
asos_snotel_UA_df.resample('12H').first().to_csv('asos_snotel_UA_12hr_df.dat', sep = ',', float_format = '%.2f')

** **

***